# Part X — Advanced Topics  
## 44. Accessibility and UX Quality (Professional Baseline) — Forms, Errors, Navigation, and Audits

Accessibility (a11y) is not “nice to have.” It’s a quality baseline in serious
products because it improves:

- usability for keyboard-only users and screen readers
- mobile usability and clarity
- error recovery and form completion rates
- overall UI consistency and professionalism
- compliance readiness (often WCAG 2.1 AA expectations)

This chapter upgrades your existing Django templates (articles/tasks/forms) into a
clean, accessible UI system with reusable patterns.

---

## 44.0 Learning Outcomes

By the end, you should be able to:

1. Explain key accessibility principles (POUR: Perceivable, Operable, Understandable, Robust).
2. Build forms that are accessible by default:
   - correct labels
   - error messages tied to fields
   - error summary for fast navigation
   - keyboard-friendly structure
3. Implement accessible global layout:
   - skip link
   - semantic landmarks (`<nav>`, `<main>`, `<header>`, `<footer>`)
   - sensible heading hierarchy (h1 → h2 → h3…)
4. Make messages/notifications accessible with `aria-live` (without breaking your UX).
5. Improve UX quality:
   - clear validation messages
   - PRG pattern + success feedback
   - loading states
   - confirm flows for destructive actions
6. Run an accessibility audit checklist (manual + automated).
7. Write basic tests to prevent regressions in form structure and error rendering.

---

## 44.1 The Accessibility “Contract” (What You’re Actually Building)

### 44.1.1 The real goal
A user should be able to:
- navigate your site using keyboard only
- understand where they are and what changed after actions
- submit forms and recover from errors quickly
- read content with assistive technologies
- use the site at different zoom levels and screen sizes

### 44.1.2 The professional baseline
Aim for WCAG 2.1 AA-style behaviors:
- adequate contrast
- keyboard accessibility
- visible focus
- descriptive labels
- meaningful error messages
- no essential interaction requiring mouse-only

You do not need to memorize WCAG rules; you need consistent patterns.

---

## 44.2 Layout and Navigation: Landmarks and “Skip to Content”

A lot of a11y comes from “basic HTML done right.”

### 44.2.1 Add a skip link (high-value, low effort)
In `templates/base.html`, place this as the first focusable element inside `<body>`:

```django
<a class="skip-link" href="#main-content">Skip to main content</a>
```

Add an id to your main content area:

```django
<main id="main-content" class="container">
  {% include "partials/_messages.html" %}
  {% block content %}{% endblock %}
</main>
```

#### Why this matters
Keyboard users often tab through navigation every page. Skip link lets them jump to
the content.

### 44.2.2 Add visible focus styles (do not remove outlines)
In `static/css/site.css`:

```css
.skip-link {
  position: absolute;
  left: -9999px;
  top: 0;
  background: #000;
  color: #fff;
  padding: 0.5rem 0.75rem;
  z-index: 9999;
}

.skip-link:focus {
  left: 0.5rem;
  top: 0.5rem;
}

:focus {
  outline: 3px solid #1a73e8;
  outline-offset: 2px;
}

:focus:not(:focus-visible) {
  outline: none;
}
```

#### Industry note
A common a11y bug is “designer removed focus outlines.” Don’t do that. If you
customize focus, keep it very visible.

### 44.2.3 Use semantic landmarks
Update your base template structure to have:

- `<header>` containing `<nav>`
- `<main>` for content
- `<footer>` for footer

You already have nav/footer partials. Make sure they use semantic tags:

`templates/partials/_nav.html`:

```django
<nav class="nav" aria-label="Primary navigation">
  ...
</nav>
```

`aria-label` is useful if there are multiple navs.

---

## 44.3 Headings and Content Structure (Screen Reader Navigation)

### 44.3.1 Heading hierarchy rule
Each page should have:
- one `<h1>` describing the page
- nested sections use `<h2>`, `<h3>`, etc.

Common bug:
- using headings purely for styling (e.g., multiple h1s)
Fix:
- use CSS for styling, keep semantic headings for structure

### 44.3.2 Example: Article detail page
Ensure:
- title is `<h1>`
- comments section uses `<h2>`

You already did something like that—good.

---

## 44.4 Forms: The Highest ROI Accessibility Work

Forms are where users fail, abandon, or get stuck. Your forms must support:

- labels correctly associated with inputs
- error messages linked to the relevant input
- an error summary at the top for quick navigation
- keyboard-friendly order
- clear required/optional cues

---

## 44.5 The “Accessible Form” Pattern (Reusable, Industry Standard)

We will implement these layers:

1. Error summary (top)
2. Field groups:
   - label
   - help text
   - input
   - error messages
3. Correct ARIA:
   - `aria-invalid="true"` when errors exist
   - `aria-describedby="..."` linking input to help text and error text

### 44.5.1 Create a reusable template partial for form fields
Create `templates/forms/_field.html`:

```django
{# Usage:
   {% include "forms/_field.html" with field=form.title label="Title" %}
#}

<div class="field {% if field.errors %}field--error{% endif %}">
  <label for="{{ field.id_for_label }}">
    {{ label|default:field.label }}
    {% if field.field.required %}
      <span class="required" aria-hidden="true">*</span>
      <span class="sr-only">(required)</span>
    {% endif %}
  </label>

  {% if field.help_text %}
    <div id="{{ field.id_for_label }}_help" class="help">
      {{ field.help_text }}
    </div>
  {% endif %}

  {# Render the widget. We will add aria attributes using widget_tweaks later
     if you want; for now we use a safe template technique: rely on Django’s default id.
  #}
  {{ field }}

  {% if field.errors %}
    <div id="{{ field.id_for_label }}_errors" class="error" role="alert">
      {% for err in field.errors %}
        <div>{{ err }}</div>
      {% endfor %}
    </div>
  {% endif %}
</div>
```

#### Add screen-reader-only utility class
In `static/css/site.css`:

```css
.sr-only {
  position: absolute;
  width: 1px;
  height: 1px;
  padding: 0;
  margin: -1px;
  overflow: hidden;
  clip: rect(0, 0, 0, 0);
  border: 0;
}
```

### 44.5.2 Add accessible styling for errors
```css
.field {
  margin: 1rem 0;
}

.field--error label {
  color: #a40000;
}

.error {
  margin-top: 0.25rem;
  color: #a40000;
}

.required {
  color: #a40000;
  margin-left: 0.25rem;
}

.help {
  color: #444;
  font-size: 0.9rem;
  margin: 0.25rem 0;
}
```

---

## 44.6 Error Summary Pattern (Critical for Accessibility and UX)

When a form has errors, users should see:
- a clear summary at top
- links that jump focus to each problematic field

Create `templates/forms/_error_summary.html`:

```django
{% if form.errors %}
  <div class="error-summary" role="alert" aria-labelledby="error-summary-title" tabindex="-1">
    <h2 id="error-summary-title">Please fix the errors below</h2>

    <ul>
      {% for field in form %}
        {% if field.errors %}
          <li>
            <a href="#{{ field.id_for_label }}">
              {{ field.label }}: {{ field.errors|first }}
            </a>
          </li>
        {% endif %}
      {% endfor %}

      {% if form.non_field_errors %}
        {% for err in form.non_field_errors %}
          <li>{{ err }}</li>
        {% endfor %}
      {% endif %}
    </ul>
  </div>
{% endif %}
```

Add CSS:

```css
.error-summary {
  border: 2px solid #a40000;
  background: #fff2f2;
  padding: 1rem;
  margin: 1rem 0;
}
```

### 44.6.1 Focus management (important)
When a form fails validation, you should move focus to the error summary so screen
reader and keyboard users land at the right place.

A minimal approach (progressive enhancement) is a tiny script in base template:

```django
<script>
  (function () {
    const el = document.querySelector(".error-summary");
    if (el) {
      el.focus();
    }
  })();
</script>
```

Why this matters:
- without focus movement, keyboard users may still be “down the page” and not see
  errors
- the error summary becomes an accessible “entry point” for error recovery

---

## 44.7 Upgrade Your Article Form Template (Using Partials)

Update `articles/templates/articles/form.html` to use the new partials.

Example structure:

```django
{% extends "base.html" %}

{% block title %}
  {% if mode == "edit" %}Edit Article{% else %}New Article{% endif %}
{% endblock %}

{% block content %}
  <h1>
    {% if mode == "edit" %}
      Edit: {{ article.title }}
    {% else %}
      New Article
    {% endif %}
  </h1>

  <form method="post" enctype="multipart/form-data" novalidate>
    {% csrf_token %}

    {% include "forms/_error_summary.html" with form=form %}

    {% include "forms/_field.html" with field=form.title label="Title" %}
    {% include "forms/_field.html" with field=form.slug label="Slug" %}
    {% include "forms/_field.html" with field=form.summary label="Summary" %}
    {% include "forms/_field.html" with field=form.body label="Body" %}
    {% include "forms/_field.html" with field=form.status label="Status" %}
    {% include "forms/_field.html" with field=form.published_at label="Published at" %}
    {% include "forms/_field.html" with field=form.cover_image label="Cover image" %}

    <div class="field">
      <div>Tags</div>
      {{ form.tags }}
      {% if form.tags.errors %}
        <div class="error" role="alert">
          {% for err in form.tags.errors %}<div>{{ err }}</div>{% endfor %}
        </div>
      {% endif %}
    </div>

    <button type="submit">
      {% if mode == "edit" %}Save changes{% else %}Create{% endif %}
    </button>
  </form>
{% endblock %}
```

### 44.7.1 Why `novalidate`?
Browsers have built-in HTML validation. Many teams disable it (`novalidate`) so:
- validation behavior is consistent across browsers
- Django form errors are the source of truth

This is a choice:
- if you keep browser validation, ensure it doesn’t conflict with server messages
- either is acceptable; consistency matters

---

## 44.8 Add Proper ARIA Attributes to Form Widgets (Optional, High Quality)

Django templates don’t easily let you add per-field attributes at render time
without helper libraries. Two clean approaches:

### Approach A (recommended): add attributes in Form `__init__`
In `articles/forms.py`:

```python
class ArticleForm(forms.ModelForm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        for name, field in self.fields.items():
            css = field.widget.attrs.get("class", "")
            field.widget.attrs["class"] = (css + " input").strip()
```

Then CSS:

```css
.input {
  width: 100%;
  max-width: 40rem;
  padding: 0.5rem;
}
```

To set `aria-describedby` for help/errors, you can do it dynamically, but it’s
hard to keep correct because errors are only known after binding.

### Approach B: use `django-widget-tweaks` (very common)
If you’re okay adding a dependency, `widget-tweaks` allows:

```django
{% load widget_tweaks %}
{{ field|add_class:"input"|attr:"aria-invalid:true" }}
```

If you want, tell me and I’ll provide a full widget-tweaks setup and a refined
field partial that sets:
- `aria-invalid` only when field has errors
- `aria-describedby` linking help and errors ids correctly

For mastery-level projects, widget-tweaks (or Django’s modern form rendering APIs)
is a standard solution.

---

## 44.9 Accessible Messages / Toasts (aria-live)

Your messages partial currently renders a list. Make it accessible by adding:

- `role="status"` for non-critical updates
- `role="alert"` for errors

Update `templates/partials/_messages.html`:

```django
{% if messages %}
  <div class="messages" aria-live="polite" aria-atomic="true">
    <ul>
      {% for message in messages %}
        <li class="message {{ message.tags }}">
          {{ message }}
        </li>
      {% endfor %}
    </ul>
  </div>
{% endif %}
```

### Why `aria-live`
It tells screen readers: “announce changes here when they appear,” which is exactly
what flash messages are.

**Note:** Don’t overuse `aria-live="assertive"`; it interrupts. Use polite for most
messages.

---

## 44.10 Buttons vs Links (Correct Semantics)

### Rule
- Use `<a>` for navigation (GET)
- Use `<button>` inside a `<form method="post">` for actions (POST)

Common bug:
- delete action done via a link (GET) → dangerous and not accessible
Correct:
- delete confirmation page (GET)
- delete submit (POST) with CSRF token

You already built confirm delete patterns—good. Ensure templates follow semantics.

---

## 44.11 Pagination Accessibility

Make pagination:
- a `<nav aria-label="Pagination">`
- clear link text
- indicate current page

Example snippet:

```django
<nav class="pagination" aria-label="Pagination">
  <p>Page {{ page_obj.number }} of {{ page_obj.paginator.num_pages }}</p>
  <ul>
    {% if page_obj.has_previous %}
      <li><a href="...">Previous</a></li>
    {% endif %}
    {% if page_obj.has_next %}
      <li><a href="...">Next</a></li>
    {% endif %}
  </ul>
</nav>
```

Avoid “Click here” links. Use descriptive labels.

---

## 44.12 Color Contrast and “Not Color Only” Signals

### 44.12.1 Contrast
Use high contrast for:
- text on backgrounds
- error messages
- links

A professional approach:
- choose a small palette
- verify with a contrast checker tool
- avoid very light gray text

### 44.12.2 Don’t rely on color alone
If errors are “red border only,” some users may not notice.

Combine:
- border + icon + text message
- error summary at top

Your error summary and error text solve this.

---

## 44.13 Keyboard Support Checklist (Manual Testing)

You should be able to:

1. Press Tab from the top and reach:
   - skip link
   - nav links
   - main content
   - forms inputs
   - submit buttons
2. See focus visibly at each step.
3. Submit form with Enter.
4. On validation error:
   - focus moves to error summary
   - each summary link jumps to the field
   - field has visible error text

This manual check catches a large portion of real issues.

---

## 44.14 Automated Accessibility Testing (Industry Tools)

Automated tools don’t replace human testing, but they catch many common mistakes.

Common tool options:
- axe (browser extension)
- pa11y
- Playwright + axe integration (very common)
- Lighthouse (Chrome) for baseline

### Recommended practice
- Run axe in local dev for key pages:
  - login
  - article form
  - task list
- Add automated a11y checks in CI for critical pages (optional; can be added later)

If you want, I can add a complete Playwright + axe CI step in a later chapter.

---

## 44.15 Tests to Prevent A11y Regressions (Practical, Not Perfect)

Accessibility is hard to fully test with unit tests, but you can prevent common
structural regressions.

### 44.15.1 Test that form errors render an error summary
In `articles/tests.py`:

```python
from django.test import TestCase
from django.urls import reverse

from articles.models import Article
from tests.factories import UserFactory


class ArticleFormA11yTests(TestCase):
    def setUp(self):
        self.user = UserFactory(username="u1")
        self.client.login(username="u1", password="pass12345")

    def test_article_form_renders_error_summary_on_invalid(self):
        url = reverse("articles:create")
        response = self.client.post(
            url,
            {
                "title": "",
                "slug": "",
                "body": "",
                "status": "draft",
            },
        )
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, 'class="error-summary"')
        self.assertContains(response, "Please fix the errors below")
```

### 44.15.2 Test that every field has a label (basic check)
You can do a simple string-based check (not perfect, but catches obvious removals):

```python
self.assertContains(response, "<label", html=False)
```

For stronger checks, use BeautifulSoup (dev dependency) and assert:
- each input id has a matching `<label for="...">`

If you want that, tell me and I’ll provide a clean BeautifulSoup test helper.

---

# 44.16 Mini‑Lab: Make Task Create/Edit Forms Accessible

Apply the same approach to `tasks/templates/tasks/form.html`:

1. Add `{% include "forms/_error_summary.html" %}`
2. Replace manual rendering with `{% include "forms/_field.html" ... %}` partials
3. Ensure delete confirm page:
   - has `<h1>`
   - uses a POST form with CSRF
   - has clear button labels (“Yes, delete task”)

Then do manual keyboard test:
- Tab through form
- Submit invalid data
- Confirm focus lands on error summary

---

## 44.17 Exercises (Do These Before Proceeding)

1. Add skip link and verify keyboard navigation works end-to-end.
2. Implement error summary + focus management for:
   - Article create/edit
   - Task create/edit
   - Login page (registration/login.html)
3. Update messages to use `aria-live` and test that messages appear after redirects.
4. Add visible focus styles and confirm they are not removed by CSS.
5. Run a manual a11y audit on three pages and write findings in `docs/a11y.md`.
6. (Optional) Add an automated a11y check tool to CI for one page.

---

## 44.18 Chapter Summary

- Accessibility is a UX and quality baseline, not an optional feature.
- The highest ROI changes are:
  - skip link + semantic landmarks
  - visible focus
  - accessible forms (labels, error summary, field error association)
  - aria-live for flash messages
- Use progressive enhancement: good HTML first, JS second.
- Manual keyboard testing catches most real problems quickly.
- Add lightweight tests to prevent regressions in error rendering and form structure.

---

Next chapter: **45. Advanced Admin and Internal Tools**  
We’ll build admin dashboards, custom admin views, performance optimizations, and
safe internal reporting tools the way professional Django teams do.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='43. internationalization_and_localization.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='45.advanced_admin_and_internal_tools.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
